#### Andrew Taylor
#### atayl136
#### EN 705.601 Applied Machine Learning
## Homework 9

In [1]:
# NIM Game code for reference

import numpy as np
from random import randint, choice

# The number of piles is 3


# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game()->list:
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(_st:list)->(int,int):
    xored = _st[0] ^ _st[1] ^ _st[2]
    if xored == 0:
        return nim_random(_st)
    for pile in range(3):
        s = _st[pile] ^ xored
        if s <= _st[pile]:
            return _st[pile]-s, pile

# Random Nim player
def nim_random(_st:list)->(int,int):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

def nim_qlearner(_st:list)->(int,int):
    global qtable
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    return move, pile  # action

Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(_a:str, _b:str):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[_a] if side == 'A' else Engines[_b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side
        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n:int, _a:str, _b:str)->(int,int):
    from collections import defaultdict
    wins = defaultdict(int)
    for _ in range(_n):
        wins[game(_a, _b)] += 1
    # info
    print(f"{_n} games, {_a:>8s}{wins['A']:5d}  {_b:>8s}{wins['B']:5d}")
    return wins['A'], wins['B']

qtable, Alpha, Gamma, Reward = None, 1.0, 0.8, 100.0

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n:int):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=np.float32)
    # play _n games
    for _ in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)  # I won
                break  # new game

            qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            
            # Switch sides for play and learning
            st1 = st2

# Equation 3 - update the qtable
def qtable_update(r:float, _st1:list, move:int, pile:int, q_future_best:float):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

### Answers to Questions

### 1. Describe the environment in the Nim learning model.

In the given Nim learning model, the environment consists of three piles, each containing a random number of items between 1 and 10 at the beginning of the game. The state of the environment is represented by a list of three integers, each representing the number of items in one of the piles. The environment is responsible for keeping track of the state and informing the agent about it. It also checks for the game termination condition, which is when all the piles are empty.

### 2. Describe the agent(s) in the Nim learning model. Is Guru an agent?

There are three types of agents in the Nim learning model:

1. **Random Agent (`nim_random`)**: This agent chooses a pile at random from the non-empty piles and removes a random number of items from it.
   
2. **Guru (`nim_guru`)**: This agent uses the mathematical solution to Nim, which involves XORing the number of items in each pile to determine the optimal move. If the XOR result is zero, it falls back to making a random move.

3. **Q-Learner (`nim_qlearner`)**: This agent uses Q-learning to choose its actions. It either exploits the best known action from its Q-table or explores a random action if it encounters a previously unvisited state.

Yes, Guru is also an agent. It has a strategy to interact with the environment, even though it's deterministic and based on mathematical principles rather than learning from experience.

### 3. Describe the reward and penalty in the Nim learning model.

In the Nim learning model, the reward structure is binary:

- **Reward**: The Q-learner receives a reward of 100.0 when it wins the game, i.e., when it removes the last item from the last non-empty pile.
  
- **Penalty**: There doesn't appear to be an explicit penalty for losing in the given code, but implicitly the agent gets a future reward of 0 in losing states, which is relatively undesirable compared to the winning reward.

The Q-value update function `qtable_update` uses these rewards to update the Q-table based on the Q-learning algorithm.

### 4. How many possible states there could be in the Nim game with a maximum of 10 items per pile and 3 piles total?

The number of possible states can be calculated as follows:

- Each pile can have between 0 and 10 items, which gives us 11 possible item counts per pile.
  
- There are 3 piles.

So, the total number of possible states would be $(11 \times 11 \times 11)$.

The total number of possible states in the Nim game, given a maximum of 10 items per pile and 3 piles in total, is 1,331.

### 5. How many possible unique actions there could be for player 1 take their first action in a NIM game with 10 items per pile and 3 piles total?

In the Nim game, an action consists of choosing a pile and then removing a certain number of items from it. 

For the first action, each of the 3 piles can be chosen. Each pile has 10 items, so a player can remove between 1 and 10 items from the chosen pile.

The number of unique actions for the first action can be calculated as follows:

$$
\text{Number of unique actions} = (\text{Number of piles}) \times (\text{Number of items that can be removed from each pile})$$


The total number of possible unique actions that player 1 could take for their first action in a Nim game, given a maximum of 10 items per pile and 3 piles in total, is 30.



### 6. Improving the Nim Game Learning Model

1. **Penalizing Losses**: One way to improve learning is to introduce a penalty for losing. This can make the Q-learning agent more sensitive to actions that lead to losing states.
   
2. **Learning Rate Decay**: Decaying the learning rate (\( \alpha \)) over time can help the Q-learning algorithm to converge more effectively.

3. **Epsilon-Greedy Strategy**: Introduce an epsilon-greedy strategy to balance exploration and exploitation, rather than just relying on the max Q-value or a random action.

### Why not use Guru in the Learning Module?

Using the Guru player inside the learning module would essentially hard-code the optimal strategy into the Q-learning agent. This would defeat the purpose of reinforcement learning, which aims to learn the optimal strategy from experience rather than having it pre-programmed.

### Coding the Solution

First, let's modify the existing `nim_qlearn` and `qtable_update` functions to incorporate the above improvements. Then we'll test the modified Q-learner against the existing Random and Guru players.


In [2]:
import numpy as np
from random import randint, choice
from collections import defaultdict

# Constants
ITEMS_MX = 10
Alpha = 1.0
Gamma = 0.8
Reward = 100.0
Penalty = -100.0  # Penalty for losing
epsilon = 0.1  # For epsilon-greedy strategy

# Initialize Q-table
qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=np.float32)

# Initialize starting position
def init_game():
    return [randint(1, ITEMS_MX), randint(1, ITEMS_MX), randint(1, ITEMS_MX)]

# Random Nim player
def nim_random(_st):
    pile = choice([i for i in range(3) if _st[i] > 0])
    return randint(1, _st[pile]), pile

# Guru Nim player
def nim_guru(_st):
    xored = _st[0] ^ _st[1] ^ _st[2]
    if xored == 0:
        return nim_random(_st)
    for pile in range(3):
        s = _st[pile] ^ xored
        if s <= _st[pile]:
            return _st[pile] - s, pile

# Enhanced Q-Learner
def nim_qlearner_enhanced(_st):
    global qtable
    if np.random.rand() < epsilon:
        # Exploration
        return nim_random(_st)
    else:
        # Exploitation
        a = np.argmax(qtable[_st[0], _st[1], _st[2]])
        move, pile = a % ITEMS_MX + 1, a // ITEMS_MX
        if move <= 0 or _st[pile] < move:
            return nim_random(_st)
        return move, pile

# Update Q-table
def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile * ITEMS_MX + move - 1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

# Learning function
def nim_qlearn_enhanced(_n):
    global qtable
    for _ in range(_n):
        st1 = init_game()
        while True:
            move, pile = nim_random(st1)  # Exploration
            st2 = list(st1)
            st2[pile] -= move
            if st2 == [0, 0, 0]:
                qtable_update(Reward, st1, move, pile, 0)
                break
            qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            st1 = st2

Engines = {'Random': nim_random, 'Guru': nim_guru, 'Enhanced_Qlearner': nim_qlearner_enhanced}

# Play the game
def game(_a, _b):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[_a] if side == 'A' else Engines[_b]
        move, pile = engine(state)
        state[pile] -= move
        if state == [0, 0, 0]:
            return side
        side = 'B' if side == 'A' else 'A'

# Play multiple games
def play_games(_n, _a, _b):
    wins = defaultdict(int)
    for _ in range(_n):
        wins[game(_a, _b)] += 1
    return wins['A'], wins['B']

# Train the enhanced Q-learner
nim_qlearn_enhanced(5000)

# Test against Random and Guru players
wins_enhanced_vs_random = play_games(1000, 'Enhanced_Qlearner', 'Random')
wins_enhanced_vs_guru = play_games(1000, 'Enhanced_Qlearner', 'Guru')

wins_enhanced_vs_random, wins_enhanced_vs_guru


((710, 290), (22, 978))

### Performance Results

After training the enhanced Q-learner, we played 1000 games against both the Random and Guru players. Here are the results:

- **Enhanced Q-learner vs Random**: 
  - Enhanced Q-learner won 710 games
  - Random won 290 games

- **Enhanced Q-learner vs Guru**: 
  - Enhanced Q-learner won 22 games
  - Guru won 978 games

### Observations

- The enhanced Q-learner performs well against the Random player, winning the majority of the games.
  
- Against the Guru player, the enhanced Q-learner still falls short, winning only 22 out of 1000 games. This is expected since the Guru player uses the optimal mathematical strategy for Nim.

While we did introduce some improvements, they didn't seem to help, the Q-learner still has difficulty beating the Guru player, which employs the mathematically optimal strategy for Nim. However, it's worth noting that the Q-learner is not trained to beat Guru but rather to learn an optimal strategy from scratch. In different settings or more complex variations of Nim, a well-trained Q-learner might perform better.
